# ⏳ Time Series Forecasting for Minute Maid Sales

This project aims to build and evaluate a time series forecasting model to predict future sales for **Minute Maid** products. By leveraging historical sales data, we apply machine learning and statistical modeling techniques to generate accurate forecasts that can support inventory planning, supply chain optimization, and data-driven business decisions.

---

### 🔍 Project Objectives:
- Load and explore Minute Maid sales time series data.
- Perform preprocessing (handling missing values, trend/seasonality decomposition).
- Build and compare forecasting models (XGBRegressor, Prophet, ElasticNet, etc.).
- Evaluate forecast accuracy using metrics like RMSE and MAPE.
- Visualize predictions and generate actionable insights.


# 1. Import Libraries

In [5]:
!pip install scikit-learn

   ---------------------------------------- 0.0/10.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/10.7 MB 8.4 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/10.7 MB 11.4 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.7 MB 13.4 MB/s eta 0:00:01
   ------------------------------------ --- 9.7/10.7 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------- 10.7/10.7 MB 11.3 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   ---------------------------------------- 0/2 [joblib]
   -------------------- --

In [7]:
!pip install prophet xgboost

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ------- -------------------------------- 2.6/13.3 MB 16.7 MB/s eta 0:00:01
   ------------------ --------------------- 6.3/13.3 MB 16.8 MB/s eta 0:00:01
   ----------------------------- ---------- 10.0/13.3 MB 17.3 MB/s eta 0:00:01
   ---------------------------------------- 13.3/13.3 MB 17.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 10.1 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 18.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 14.1 MB/s eta 0:0

In [10]:
import os
import shutil
import glob
from azureml.opendatasets import OjSalesSimulated
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit
from prophet import Prophet
from xgboost import XGBRegressor

# 2. Open Datasets

# 3. Extract Data for Minute Maid

# 4. Getting the Data Ready

# 5. Data Partition

# 6. Data Normalization

# 7. Model Initialization

# 8. Model Hyperparameter Tuning

# 9. Model Training

# 10. Model Metrics Final outputs